In [1]:
import pandas as pd
import json
import os
import googlemaps
import numpy as np

class VinMartStore:
    def __init__(self, xlsx):
        self.xlsx = xlsx
        self.warehouse_coord = (21.078295, 105.966889)
        
    def read(self):
        self.data = pd.read_excel(self.xlsx)
        self.lats = [self.warehouse_coord[0]] + list(self.data.correct_lat)
        self.lngs = [self.warehouse_coord[1]] + list(self.data.correct_lng)
        self.coordinates = list(zip(self.lats, self.lngs))
        self.vinmart_lats = self.data.correct_lat
        self.vinmart_lngs = self.data.correct_lng
        self.covered_point = self.get_covered_point()
        
    def get_covered_point(self):
        covered_lat = np.min(self.vinmart_lats) + ((np.max(self.vinmart_lats) - np.min(self.vinmart_lats))/ 2)
        covered_lng = np.min(self.vinmart_lngs) + ((np.max(self.vinmart_lngs) - np.min(self.vinmart_lngs))/ 2)
        return covered_lat, covered_lng


class StaticVinMartStore:
    def __init__(self, vinmart_store, static_folder='./static'):
        self.vinmart_data = VinMartStore(vinmart_store)
        self.static_folder = static_folder
        self.gg_client = googlemaps.Client(key=os.getenv('GOOGLE_API_KEY'))
        self.check()
            
    def parse_into_np_matrix(self, rows, size, dtype='distance'):
        np_matrix = np.zeros(size)
        for i, row in enumerate(rows):
            for j in range(len(row['elements'])):
                np_matrix[i][j] = row['elements'][j][dtype]['value']
        return np_matrix
    
    def get_matrix(self, google_matrix):
        if google_matrix['status'] == 'OK':
            size = (len(google_matrix['origin_addresses']), len(google_matrix['destination_addresses']))
            distance_matrix = self.parse_into_np_matrix(google_matrix['rows'], size)
            return distance_matrix

    def get_utility_matrices(self, origins, destinations, mode='driving'):
        if len(origins) * len(destinations) < 100:
            gg_mat = self.gg_client.distance_matrix(origins=origins, destinations=destinations, mode=mode)
            return self.get_matrix(gg_mat)
        else:
            dist = None
            for i, origin in enumerate(origins):
                gg_matrix = self.gg_client.distance_matrix(origins=origin, destinations=destinations, mode=mode)
                distance_matrix = self.get_matrix(gg_matrix)
                if i == 0:
                    dist = distance_matrix
                else:
                    dist = np.concatenate((dist, distance_matrix), axis=0)
            return dist
        
    def get_direction_description(self):
        dirdes_json = {}
        for i, origin in enumerate(self.vinmart_data.coordinates):
            dirdes_json[str(i)] = {}
            for j, target in enumerate(self.vinmart_data.coordinates):
                directions_result = self.gg_client.directions(origin, target, mode="driving")
                dirdes_json[str(i)][str(j)] = directions_result
        return dirdes_json
        
    def check(self):
        self.vinmart_data.read()
        total = len(self.vinmart_data.coordinates)
        static_files = os.listdir(self.static_folder)
        # Check coordinate.csv file
        if 'coordinate.csv' not in static_files:
            coord_file = open(os.path.join(self.static_folder, 'coordinate.csv'), 'w')
            for coord in self.vinmart_data.coordinates:
                coord_file.write('{},{}\n'.format(coord[0], coord[1]))
            coord_file.close()
        else:
            coord_file = open(os.path.join(self.static_folder, 'coordinate.csv'), 'r')
            lines = coord_file.readlines()
            if len(lines) != total:
                raise Exception('Mismatched between previous data and newly data, please check storages coordinate data')
        # Check distance_matrix.csv file
        if 'distance_matrix.txt' not in static_files:
            dist_mat_file = open(os.path.join(self.static_folder, 'distance_matrix.txt'), 'w')
            dist_mat = self.get_utility_matrices(self.vinmart_data.coordinates, self.vinmart_data.coordinates)
            np.savetxt(dist_mat_file, dist_mat, fmt='%.0f')
            dist_mat_file.close()
        # Check direction.json file
        if 'direction.json' not in static_files: 
            direction_json = self.get_direction_description()
            json.dump(direction_json, open(os.path.join(self.static_folder, 'direction.json'), 'w'))
        # Check direction.json file
        if 'geocode.json' not in static_files: 
            geocode_json = {i: self.gg_client.reverse_geocode(latlng)[0] for i, latlng in enumerate(self.vinmart_data.coordinates)}
            json.dump(geocode_json, open(os.path.join(self.static_folder, 'geocode.json'), 'w'))
            
    
class VinMartProblemReader:
    def __init__(self, problem_file):
        self.problem_file = problem_file
        self.problem = json.load(open(problem_file, 'r'))
        
    def read_tsptw(self):
        et = np.array(self.problem['ET_i']) * 60 # default mins
        lt = np.array(self.problem['LT_i']) * 60
        time_window = list(zip(et.tolist(), lt.tolist()))
        time_window = time_window[-1:] + time_window[1:-1]
        idx_locs = self.problem['Location']
        idx_locs = np.array(idx_locs) + 1
        idx_locs = [0] + idx_locs.tolist()
        self.vinmart_ids = idx_locs
        self.vinmart_tw = time_window

    
class VinMartProblemParser:
    def __init__(self, vinmart_store, problem_file, static_folder='./static'):
        self.vinmart_data = VinMartStore(vinmart_store)
        self.vinmart_problem = VinMartProblemReader(problem_file)
        self.static_folder = static_folder
        
    def parse(self, format='json', output=None):
        if format == 'json':
            self.vinmart_data.read()
            self.vinmart_problem.read_tsptw()
            self.coordinates = [self.vinmart_data.coordinates[i] for i in self.vinmart_problem.vinmart_ids]
            self.geocodes = self.get_geocode()
            self.save_json(output)
            
    def get_geocode(self):
        if 'geocode.json' not in os.listdir(self.static_folder):
            self.gg_client = googlemaps.Client(key=os.getenv('GOOGLE_API_KEY'))
            return [self.gg_client.reverse_geocode(latlng)[0] for latlng in self.coordinates]
        else:
            geocode_json = json.load(open(os.path.join(self.static_folder, 'geocode.json'), 'r'))
            return [geocode_json[str(i)] for i in self.vinmart_problem.vinmart_ids]
        
    def save_json(self, output=None):
        json_data = {}
        json_data['problem'] = {'depot': 0,
                                'destination': self.geocodes,
                                'time_window': self.vinmart_problem.vinmart_tw}
        json_data['metadata'] = {'visualize': {'covered_point': self.vinmart_data.covered_point,
                                               'zoom_level': 13,
                                               'layout': {'width': '1400px', 
                                                          'height': '800px'}}}
        if output is None:
            output = self.vinmart_problem.problem_file
            output = output[output.rfind('/') + 1:output.rfind('.')]
        output += '_tsptw.json'
        json.dump(json_data, open(output, 'w'))
        self.output_file = output
        
def load_static_data(static_folder='./static'):
    coordinates = pd.read_csv(os.path.join(static_folder, 'coordinate.csv'))
    directions = json.load(open(os.path.join(static_folder, 'direction.json')))
    distance_matrix = np.loadtxt(open(os.path.join(static_folder, 'distance_matrix.txt')))
    geocodes = json.load(open(os.path.join(static_folder, 'geocode.json')))
    return {'coordinate': coordinates, 
            'direction': directions, 
            'distance_matrix': distance_matrix, 
            'geocode': geocodes}

def execute(filename, metric='distance', desc=True, layout=None):
    assert metric == 'duration', "Metric of TSPTW must be duration"
    file_data = '../data/tw_data/{}.txt'.format(filename)
    json_filename = os.path.join('..', 'notebook', 'json_data', filename + '_tsptw.json')
    if not os.path.exists(json_filename):
        prob = VinMartProblemParser('../data/Location_BaDinh.xlsx', '../data/tw_data/{}.txt'.format(filename))
        prob.parse(output='./json_data/%s' % filename)
    static_data = load_static_data()
    tsptw = TSPTW(static_data, metric=metric)
    tsptw.read(json_filename)
    tsptw.run()
    vis = TSPTWVis(tsptw, rps=0.2, layout=layout)
    stdout = None
    if desc is True:
        stdout, routes, route_desc = vis.print_readable_description()
    figure, route_layer_apis = vis.draw_figure(save=False)
    return {'figure': figure, 
            'stdout': stdout, 
            'route_layer_api': route_layer_apis,
            'route': routes,
            'route_description': route_desc}


In [2]:
prob = StaticVinMartStore('../data/Location_BaDinh.xlsx')